# Hyperparameter Optimization - Optuna

Optuna is an open source hyperparameter optimization framework to automate hyperparameter search. It uses a **Bayesian Optimization** algorithm to efficiently search for the best hyperparameters, based on the results of previous trials. 

Hyperparameters are the parameters set for an algorithm before the learning process takes place.
Such parameters have a direct effect on how well a model can learn from its environment. To
reach maximum potential for learning in an environment, a model must have its hyperparameters
optimized, and there are several ways to do this. 

For this project, we investigated Optuna, which allows one to set an interval between certain values of hyperparameters and, in doing so, automatically find the best ones. The table below contains all hyperparameters used in the project. However, several more hyperparameters are available for each algorithm.

## Hyperparameter Search Space

| Hyperparameter | Range of Values |
|-----------------|--------|
| **Learning Rate** | (0.001, 0.1) |
| **Batch Size** | [32, 64, 128] |
| **Replay Memory Size** | (1000, 10000) |
| **Discount Factor** | (0.1, 0.99) |
| **Frame Repeat** | [4, 8, 12, 16, 20] |
| **Epsilon Decay** | (0.9900, 0.9999) |


## Hyperparameters

- **Learning Rate**: Learning rate is the rate at which the model learns and adapts to a problem or environment. It
is the degree to which a new value is accepted versus an old value.

<br>

- **Batch Size**: Batch size defines the number of samples that will be propagated through the neural network
prior to an update of the internal model parameters used. It impacts how well a model learns
and has a direct effect on the stability of a learning process of an agent.

<br>

- **Replay Memory Size**: Replay memory size refers to the number of past experiences or transitions stored in the memory of a reinforcement learning system. It is a hyperparameter that can impact the learning process and performance of the algorithm, and its appropriate size depends on various factors.

<br>

- **Discount Factor**: The discount factor is a value used in reinforcement learning algorithms that determines the importance of future rewards. It is a number between 0 and 1, where 0 means the agent only cares about immediate rewards and 1 means it cares equally about all future rewards. A discount factor closer to 1 gives more weight to future rewards, and a lower discount factor prioritizes immediate rewards. The discount factor is used to calculate the expected reward that an agent will receive over time and is an important parameter in many reinforcement learning algorithms, such as Q-learning and policy gradient methods.

<br>

- **Frame Repeat**: Frame repeat is a parameter used in some reinforcement learning algorithms, particularly in those that use deep neural networks to approximate the optimal policy or value function. It refers to the number of times that an action is repeated before the agent receives a new observation or state from the environment. By repeating the action for several frames, the agent can process more information and make more informed decisions. However, this comes at the cost of increasing the computational complexity and training time. The appropriate value of the frame repeat parameter depends on the characteristics of the environment and the learning algorithm being used.

<br>

- **Epsilon Decay**: Epsilon decay is a technique used in reinforcement learning to gradually decrease the exploration rate, epsilon, over time. Epsilon is the probability that the agent takes a random action instead of the action with the highest predicted value. During the early stages of training, a high epsilon value is used to encourage the agent to explore the environment and learn about the possible actions. As training progresses, the agent's policy becomes more refined, and the exploration rate is gradually reduced. Epsilon decay can be implemented in different ways, such as linear or exponential decay, and the appropriate decay rate depends on the specific learning problem and the algorithm being used.


## Import the required libraries

In [27]:
import itertools as it
import os
import random
from collections import deque
from time import sleep, time
import numpy as np
import skimage.transform
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import trange
import vizdoom as vzd
import torchvision.models as models
import optuna

import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from models import *
from main import train_agent

## Define the objective function

In [ ]:
def objective(trial: Trial) -> float:
    # Define the hyperparameters to tune
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    replay_memory_size = trial.suggest_int("replay_memory_size", 1000, 10000, step=1000)
    discount_factor = trial.suggest_float("discount_factor", 0.1, 0.99)
    frame_repeat = trial.suggest_categorical("frame_repeat", [4, 8, 12, 16, 20])
    optim_ = trial.suggest_categorical("optimizer", ["SGD", "Adam"])
    epsilon_decay = trial.suggest_float("epsilon_decay",0.9900,0.9999)
    
     # Print the hyperparameters being currently tested
    print(f"\nLearning rate: {learning_rate}, Batch size: {batch_size}, Replay memory size: {replay_memory_size},\n"
          f"Discount factor: {discount_factor}, Frame repeat: {frame_repeat}, Optimizer: {optim_}, \n"
          f"Epsilon decay: {epsilon_decay}\n")


    # Initialize game and actions
    game = create_simple_game()
    n = game.get_available_buttons_size()
    actions = [list(a) for a in it.product([0, 1], repeat=n)]

    # Initialize our agent with the set hyperparameters
    agent = DDQNAgent(
        len(actions),
        lr=learning_rate,
        batch_size=batch_size,
        memory_size=replay_memory_size,
        discount_factor=discount_factor,
        epsilon_decay = epsilon_decay,
        optimizer=optim_,
        load_model = load_model
    )

    # Run the training for the set number of epochs
    agent, game,test_scores = train_agent(
        game,
        agent,
        actions,
        num_epochs=10,
        frame_repeat=frame_repeat,
        steps_per_epoch=2000,
    )

    return test_scores.mean()

# Set up the Optuna study
study = optuna.create_study(direction="maximize", sampler=TPESampler())

# Run the optimization
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and the corresponding score
print("Best hyperparameters: ", study.best_params)
print("Best score: ", study.best_value)

[I 2023-05-03 23:21:35,206] A new study created in memory with name: no-name-6cc88990-098d-4249-b069-ccbdbf0596c3



Learning rate: 0.010820983255735073, Batch size: 128, Replay memory size: 8000,
Discount factor: 0.5681421501464371, Frame repeat: 8, Optimizer: SGD, 
Epsilon decay: 0.9981163155136186

Initializing doom...
Doom initialized.
Initializing New Model

Epoch #1


Results: mean: -146.5 +/- 174.5, min: -365.0, max: 70.0,

Testing...


Results: mean: -113.8 +/- 161.4, min: -360.0 max: 66.0
Total elapsed time: 2.96 minutes

Epoch #2


Results: mean: -172.4 +/- 161.1, min: -345.0, max: 70.0,

Testing...


...

Results: mean: 29.7 +/- 66.0, min: -335.0, max: 66.0,

Testing...


Results: mean: 23.0 +/- 90.7, min: -350.0 max: 66.0
Total elapsed time: 34.54 minutes

Epoch #9


Results: mean: -11.5 +/- 95.0, min: -355.0, max: 66.0,

Testing...


Results: mean: -41.9 +/- 120.3, min: -360.0 max: 66.0
Total elapsed time: 40.85 minutes

Epoch #10


 59%|██████████████████████████████████████████████▎                               | 1187/2000 [02:28<01:33,  8.69it/s]

## Best Hyperparameters

| Hyperparameter | Range of Values |
|-----------------|--------|
| **Learning Rate** | 0.007124271448356194 |
| **Batch Size** | 128 |
| **Replay Memory Size** | 5000 |
| **Discount Factor** | 0.8828099164309334 |
| **Frame Repeat** | 12 |
| **Epsilon Decay** | 0.9956338864091042 |
